<a href="https://colab.research.google.com/github/Dmitrijg24k/-NeuralNetwork/blob/main/HappyOrSad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2021-02-06 18:54:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-02-06 18:55:00 (106 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy1-05.png', 'happy2-07.png', 'happy1-07.png', 'happy2-06.png', 'happy1-16.png', 'happy2-09.png', 'happy2-13.png', 'happy2-02.png', 'happy2-18.png', 'happy1-13.png']
['sad2-09.png', 'sad1-08.png', 'sad1-00.png', 'sad2-07.png', 'sad2-10.png', 'sad2-17.png', 'sad2-13.png', 'sad2-11.png', 'sad2-08.png', 'sad1-05.png']


In [16]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
          if(logs.get('acc')>=DESIRED_ACCURACY):
              print("Достигнута точность 99,9%, поэтому обучение закончено!")
              self.model.stop_training = True

    callbacks = myCallback()
    
    # Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])
        

     # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
      # и train_generator, вызовом train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)
    validation_datagen = ImageDataGenerator(rescale=1/255)
    # Используйте target_size  150 X 150.
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/', 
        target_size=(150, 150),  
        batch_size=128,
        class_mode='binary'
    )

    history= model.fit(train_generator, epochs=25, callbacks=[callbacks])
    # model fitting
    return history.history['acc'][-1]

In [18]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/25
1/1 [==============================] - 2s 2s/step - loss: 0.6931 - acc: 0.5000
Epoch 2/25
1/1 [==============================] - 2s 2s/step - loss: 8.5674 - acc: 0.5000
Epoch 3/25
1/1 [==============================] - 2s 2s/step - loss: 2.1380 - acc: 0.5000
Epoch 4/25
1/1 [==============================] - 2s 2s/step - loss: 0.6931 - acc: 0.5000
Epoch 5/25
1/1 [==============================] - 2s 2s/step - loss: 0.6657 - acc: 0.5000
Epoch 6/25
1/1 [==============================] - 2s 2s/step - loss: 0.6678 - acc: 0.5000
Epoch 7/25
1/1 [==============================] - 2s 2s/step - loss: 0.6352 - acc: 0.5000
Epoch 8/25
1/1 [==============================] - 2s 2s/step - loss: 0.6227 - acc: 0.5000
Epoch 9/25
1/1 [==============================] - 2s 2s/step - loss: 0.5357 - acc: 0.6625
Epoch 10/25
1/1 [==============================] - 2s 2s/step - loss: 0.4976 - acc: 0.8875
Epoch 11/25
1/1 [==============================] - 2s 2s/st

1.0